# My Second Notebook for this competition

I used XGBRegressor for this project on my first notebook.
https://www.kaggle.com/satoshiss/spam-detection-tabular-playground-nov-2021

**Try Neural Networks**
When I read through discussion board, I saw someone mentioned NN works well for this dataset.
https://www.kaggle.com/c/tabular-playground-series-nov-2021/discussion/284784

I wanted to practice neural networks, so I will give it a try. I refferred to Intro to Deep Learning
on the Kaggle course and also the notebook(https://www.kaggle.com/ryanholbrook/detecting-the-higgs-boson-with-tpus).


In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
# TensorFlow
import tensorflow as tf
print("Tensorflow version " + tf.__version__)

# Detect and init the TPU
try: # detect TPUs
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver.connect() # TPU detection
    strategy = tf.distribute.TPUStrategy(tpu)
except ValueError: # detect GPUs
    strategy = tf.distribute.get_strategy() # default strategy that works on CPU and single GPU
print("Number of accelerators: ", strategy.num_replicas_in_sync)
    
# Plotting
import pandas as pd
import matplotlib.pyplot as plt

# Matplotlib defaults
plt.style.use('seaborn-whitegrid')
plt.rc('figure', autolayout=True)
plt.rc('axes', labelweight='bold', labelsize='large',
       titleweight='bold', titlesize=18, titlepad=10)


# Model
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras import callbacks

In [ ]:
#Load Data

df = pd.read_csv("../input/tabular-playground-series-nov-2021/train.csv")
df_test = pd.read_csv("../input/tabular-playground-series-nov-2021/test.csv")


In [ ]:
from sklearn import preprocessing

df_train = df.sample(frac=0.7, random_state=0)
df_valid = df.drop(df_train.index)


# Split features and target
X_train = df_train.drop(['target','id'],axis=1)
X_valid = df_valid.drop(['target','id'],axis=1)
y_train = df_train['target']
y_valid = df_valid['target']



In [ ]:

df_test = df_test.drop('id',axis=1)
scaler = preprocessing.StandardScaler()

X_train[:] = scaler.fit_transform(X_train[:])
X_valid[:] =scaler.transform(X_valid[:])
df_test[:] = scaler.transform(df_test[:])

In [ ]:
from tensorflow import keras
from tensorflow.keras import layers, callbacks

early_stopping = callbacks.EarlyStopping(
    min_delta=0.001, # minimium amount of change to count as an improvement
    patience=10, # how many epochs to wait before stopping
    restore_best_weights=True,
)

model = keras.Sequential([
    layers.Dense(4,activation ='relu', input_shape =[100],
    #kernel_regularizer= keras.regularizers.l2(0.005)
                ),
    layers.Dense(4,activation='softmax'),
    layers.Dense(1,activation='sigmoid'),
    
])

In [ ]:
model.compile(
     optimizer ='adam',
     loss='binary_crossentropy',
     metrics =['binary_accuracy']
    )

In [ ]:
history = model.fit(
X_train,y_train,
validation_data =(X_valid,y_valid),
batch_size=512,
epochs =1000,
callbacks=[early_stopping],
verbose =0
)



In [ ]:
history_df = pd.DataFrame(history.history)
# Start the plot at epoch 5
history_df.loc[5:, ['loss', 'val_loss']].plot()
history_df.loc[5:, ['binary_accuracy', 'val_binary_accuracy']].plot()

print(("Best Validation Loss: {:0.4f}" +\
      "\nBest Validation Accuracy: {:0.4f}")\
      .format(history_df['val_loss'].min(), 
              history_df['val_binary_accuracy'].max()))

In [ ]:
index = pd.read_csv("../input/tabular-playground-series-nov-2021/sample_submission.csv")
predictions=model.predict(df_test)
data = pd.DataFrame(data=predictions,columns=['target'])
output =pd.DataFrame({'id': index.id,'target':data['target']})
output.to_csv('submission.csv',index=False)
